In [ ]:
import os 
# 보안을 위해 따로 만든 .env파일을 불러올 수 있게 해주는 라이브러리
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
# "OPENAI_API_KEY"라는 이름의 환경 변수 값을 가져옴
open_ai_key = os.getenv('OPENAI_API_KEY')
# 웬만하면 하드코딩하지 않는 것이 좋음
client = OpenAI(api_key= 'sk-proj-IijgVfhp9Yd2dAYhmtEpT3BlbkFJah84IhiGypsfWzyuAOpV')

## API Key 작동 확인

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='sk-proj-IijgVfhp9Yd2dAYhmtEpT3BlbkFJah84IhiGypsfWzyuAOpV')

# 답변 받을 수 있는 함수 제작
def get_response(prompt):
    # completions endpoint를 사용함
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt},],
    )
    # response는 JSON형태로 반환됨
    return response.choices[0].message.content.strip()

In [ ]:
response = get_response("하 힘들다")
response

In [ ]:
# LLM의 다양한 자연어 처리 작업을 수행할 수 있도록 도움
from llama_index.llms.openai import OpenAI as OpenAI_llama
# ChatMessage는 텍스트를 생성하는 기능을 제공
from llama_index.core.llms import ChatMessage

# 새로운 객체 생성
# 클라이언트 초기화
client_llama = OpenAI_llama(model='gpt-3.5-turbo')

# 프롬프트를 이용하여 텍스트 생성
# 대화 시스템에 더욱 가까움
# 이전 대화를 기억하게 할 수 있음
messages = [ChatMessage(role="user", content="안녕! 넌 누구야?")]
response = client_llama.chat(messages)
response.message.content

## LLM

### raw LLM

In [ ]:
query = "OpenAI의 sora 모델에 대해서 설명해줘."

# 질문에 대해 단순히 답변을 받음
resp = get_response(query)
print(resp)

### LLM with context

In [ ]:
# context를 주고 그 context에서 정보를 찾도록 하는 코드
# f string을 이용함
prompt = f"""
Utilizing the given context, please answer the user question.

[context]
Text-to-video
Sora
Main article: Sora (text-to-video model)
Sora is a text-to-video model that can generate videos based on short descriptive prompts[193] as well as extend existing videos forwards or backwards in time.[194] It can generate videos with resolution up to 1920x1080 or 1080x1920. The maximal length of generated videos is unknown.

The team that developed Sora named it after the Japanese word for sky to signify its "limitless creative potential".[193] The technology behind Sora is an adaptation of the technology behind the DALL·E 3 text-to-image model.[195] OpenAI trained the system using publicly-available videos as well as copyrighted videos licensed for the purpose, but did not reveal the number or the exact source of the videos.[193]

OpenAI demonstrated a few Sora-created high-definition videos to the general public on February 15, 2024, stating that the technology was able to generate videos up to one minute long. It also shared a technical report highlighting the methods used to train the model and the model's capabilities.[195] It acknowledged some shortcomings of the system, including struggles in simulating complex physics.[196] Will Douglas Heaven of the MIT Technology Review called the demonstration videos "impressive", but noted that they must have been cherry-picked and may not be representative of Sora's typical output.[195]


[user question]
{query}
"""

resp = get_response(prompt)
print(resp)

In [ ]:
query = "OpenAI의 sora 모델에 대해서 설명해줘."
prompt = f"""
Utilizing the given context, please answer the user question.

[context]
OpenAI Global LLC then announced its intention to commercially license its technologies.[47] It planned to spend the $1 billion "within five years, and possibly much faster."[48] Altman has stated that even a billion dollars may turn out to be insufficient, and that the lab may ultimately need "more capital than any non-profit has ever raised" to achieve artificial general intelligence.[49]


[user question]
{query}
"""
# 찾는 정보가 context에 없다면 모른다고 출력함
resp = get_response(prompt)
print(resp)

### 데이터 마련하기

In [ ]:
# 웹에서 데이터를 가져올 수 있도록 해주는 라이브러리
import requests

# get 함수를 사용하여 위키피디아의 데이터를 가져옴
response = requests.get(
    "https://en.wikipedia.org/w/api.php",
    params={
        # 수행해야 할 작업
        "action": "query",
        # 형식은 JSON
        "format": "json",
        "titles": "OpenAI",
        # 요약
        "prop": "extracts",
        "explaintext": True,
    },
).json()


In [ ]:
response
response['query']['pages']['48795986']['extract'].split('\n')

In [ ]:
# 위 코드와 동일함
# iterator는 index와 달리 데이터 내부 구조의 제약이 없음
page = next(iter(response["query"]["pages"].values())) # {'pageid': ...} 형태의 딕셔너리 반환
text = page["extract"]
text.split('\n')

### indexing
### chunk의 개념 이해하기

In [ ]:
# 대규모 데이터 집합을 처리하기 위해 chunk를 사용
def get_chunks(text, chunk_size):
    # 공백을 기준으로 text를 나누고 words리스트에 저장함
    words = text.split()
    # ' '는 단어 사이의 구분자 역할
    # chunk_size 개수 만큼의 단어들이 한 문장이 되어 chunk에 저장된다.
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

    return chunks

In [ ]:
text = """Sora
Main article: Sora (text-to-video model)
Sora is a text-to-video model that can generate videos based on short descriptive prompts[193] as well as extend existing videos forwards or backwards in time.[194] It can generate videos with resolution up to 1920x1080 or 1080x1920. The maximal length of generated videos is unknown.

The team that developed Sora named it after the Japanese word for sky to signify its "limitless creative potential".[193] The technology behind Sora is an adaptation of the technology behind the DALL·E 3 text-to-image model.[195] OpenAI trained the system using publicly-available videos as well as copyrighted videos licensed for the purpose, but did not reveal the number or the exact source of the videos.[193]

OpenAI demonstrated a few Sora-created high-definition videos to the general public on February 15, 2024, stating that the technology was able to generate videos up to one minute long. It also shared a technical report highlighting the methods used to train the model and the model's capabilities.[195] It acknowledged some shortcomings of the system, including struggles in simulating complex physics.[196] Will Douglas Heaven of the MIT Technology Review called the demonstration videos "impressive", but noted that they must have been cherry-picked and may not be representative of Sora's typical output.[195]"""

chunks = get_chunks(text, 128)
chunks

In [ ]:
# "안녕\n나는"이라는 텍스트를 임베딩하여 벡터로 반환해줌
client.embeddings.create(input = "안녕\n나는", model="text-embedding-3-small")

In [ ]:
# 임베딩 값을 반환하는 함수
def get_embedding(text):
   return client.embeddings.create(input = [text], model="text-embedding-3-small").data[0].embedding # [0.02429252117872238, -0.05541412532329559]처럼 리스트로 표현됨

In [ ]:
# chunks의 각 문장들을 모두 임베딩시켜서 출력함
embeddings = [get_embedding(c) for c in chunks]
embeddings

### retrieve

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# query는 사용자가 입력하는 문장
# top_k는 사용자가 입력한 문장과 유사한 문장 중, 출력할 문장의 개수
# db_embedding은 db에 있는 문장들을 임베딩 해둔 리스트
def retriever(query, top_k, db_embedding):
    # 사용자가 입력한 query를 임베딩하여 q_emb 변수에 저장함
    q_emb = get_embedding(query)

    # q_emb를 리스트로 만들고 db_embedding과의 코사인 유사도를 구함
    # 벡터의 차원만 같다면, 행렬의 크기는 달라도 괜찮다.
    sim_score = cosine_similarity([q_emb], db_embedding)[0]
    # 내림차순으로 정리하고, 유사도가 큰 문장을 top_k개 만큼 저장
    # argsort 함수는 오름차순으로 '인덱스'를 반환함
    max_indices = np.argsort(sim_score)[::-1][:top_k] # 인덱스 값

    # i는 index 변수
    retrieved_datas = [chunks[i] for i in max_indices]

    return retrieved_datas



In [ ]:
# embeddings는 위에서 만든 벡터 임베딩화한 데이터셋
contexts = retriever("OpenAI의 sora 모델에 대해 설명해줘", 3, embeddings) # LLM에 전달할 유사한 문장 3개를 출력해보는 코드
contexts

### Generate

In [ ]:
# query는 사용자가 입력한 문장
# contexts는 이미 가지고 있는 데이터셋을 뜻함(리스트)
def generator(query, contexts):
    # 줄 바꿈을 구분자로 정하고 contexts에 있는 요소들을 합침
    # 아까 출력한 3개의 문장들을 하나의 context로 합치는 코드
    context = "\n".join(contexts)

    prompt = f"""
    Utilizing the given context, please answer the user question.

    [context]
    {context}

    [user question]
    {query}
    """

    return get_response(prompt)


In [ ]:
generator(query, contexts)

# Llamaindex
## loading data

In [ ]:
# 경로 조작 및 파일 시스템 작업을 직관적으로 처리
from pathlib import Path

# expanduser() 메서드는 경로에 ~(틸드)가 포함된 경우 이를 사용자 홈 디렉토리로 확장해 준다. 하지만 이 예제에서는 경로에 ~가 없으므로 별다른 변화는 없다.
data_path = Path(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디").expanduser()

# openai.txt 에 넣고 싶은 텍스트
text = "w3lcom3003"

# 만약 data_path가 존재하지 않으면, 해당 디렉토리를 생성함
if not data_path.exists():
    Path.mkdir(data_path)

# with는 작업이 끝나면 자동으로 파일을 닫아줌
# openai.txt 파일을 쓰기 전용으로 연다.
# 파일이 없으면 생성, 있다면 기존 내용을 덮어 쓴다.
with open(f"{data_path}/openai.txt", "w") as fp:
    fp.write(text)

In [ ]:
# VectorStoreIndex는 문서의 벡터 인덱스를 생성하고 관리함
# SimpleDirectoryReader는 디렉토리나 파일에서 데이터를 읽어오는 클래스이다.
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

file_path = r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디"

# load_data()는 데이터를 읽어와 문서 객체로 변환함
documents = SimpleDirectoryReader(file_path).load_data()
# chunk로 분할, 벡터로 변환, 인덱싱
# from_documents의 인자에 원하는 텍스트 분할기를 줘서 chunk_size를 조절할 수 있음 
vector_index = VectorStoreIndex.from_documents(documents) # chunk_size 1024 
print(documents)

In [ ]:
# 위에서 생성된 vector_index로부터 쿼리 엔진을 생성
# query engine이란 query를 실행하고 결과를 반환해주는 소프트웨어 컴포넌트
query_engine = vector_index.as_query_engine()
# 질문에 대한 응답 받기
response = query_engine.query("OpenAI의 sora 모델에 대해서 알려줘")
print(response)

# 쿼리 엔진에서 생성된 프롬프트를 출력함
# 프롬프트의 구조를 파악 가능
prompt = query_engine.get_prompts()
print(prompt)

In [ ]:
from llama_index.core import Settings
Settings

In [ ]:
Settings._llm

In [ ]:
Settings._embed_model

In [ ]:
Settings._node_parser

### Chunk size 조절하기

In [ ]:
# 문장을 쪼개기 위해 SentenceSplitter 클래스 호출
from llama_index.core.node_parser import SentenceSplitter

# chunk_overlap을 통해 일부러 단어들을 겹치게 하여 데이터 손실을 줄인다.
text_splitter = SentenceSplitter(chunk_size=128,
                                chunk_overlap = 10)

# 문서를 청크로 나눈다. (노드)
nodes = text_splitter.get_nodes_from_documents(documents)
len(nodes)

In [ ]:
nodes[0]

In [ ]:
nodes

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

# 메타데이터 없이 출력
text_splitter = TokenTextSplitter(chunk_size=128, # 기본값을 4000토큰
                                chunk_overlap = 10, # 기본값은 200토큰
                                include_metadata=False)

# documents를 클래스 객체 안의 함수를 통해 쪼개서 노드를 생성
nodes = text_splitter.get_nodes_from_documents(documents)
len(nodes)

In [ ]:
nodes[0]

In [ ]:
nodes

### Append node to vector index

In [ ]:
# TextNode 클래스는 텍스트 데이터를 구조화할 수 있도록 도와준다
from llama_index.core.schema import TextNode

# 텍스트에 대한 노드를 생성
node_new = TextNode(text = "OpenAI의 소라는 text to video 모델로, 생성할 수 있는 비디오는 오직 2d 애니메이션이다. 대표적인 창작물로는 NLP에 대한 소개 애니메이션이 있다.")

nodes = []

nodes.append(node_new)

print(nodes)

In [ ]:
# nodes는 벡터 데이터의 모음
vector_index = VectorStoreIndex(nodes)
# 벡터 데이터에 대해 쿼리 엔진 생성
query_engine = vector_index.as_query_engine()
# 쿼리 실행
response = query_engine.query("OpenAI의 sora 모델에 대해서 알려줘")
response.response

### Saving vector index

In [ ]:
# vector_index = VectorStoreIndex(nodes[:-1])

# 현재 벡터 인덱스 상태를 파일 시스템에 저장한다.
vector_index.storage_context.persist(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디")


In [ ]:
response = get_response("Mistral AI는 어느 국적의 회사야?")
response

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# 주어진 경로의 파일 데이터를 불러옴
documents = SimpleDirectoryReader(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디").load_data()
# 불러온 파일 데이터를 벡터 인덱스로 바꿈
vector_index = VectorStoreIndex.from_documents(documents)
# 쿼리 엔진 실행
query_engine = vector_index.as_query_engine()
response = query_engine.query("Mistral AI는 어느 국적의 회사야?")
print(response.response)

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# persist인자를 통해 저장 디렉토리를 지정
storage_context = StorageContext.from_defaults(persist_dir=r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디")
# 저장된 벡터 인덱스를 읽어 들인다.
vector_index = load_index_from_storage(storage_context)

## Retriever

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

# 임베딩 모델과 벡터 차원을 지정
embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=128)
li = list(embed_model.get_text_embedding('머신러닝은 멋져'))
len(li) # 128 차원의 개수

In [ ]:
Settings.embed_model = embed_model
vector_index = VectorStoreIndex(nodes)

In [ ]:
Settings.embed_model

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever

retriever = VectorIndexRetriever(
    # 검색에 사용할 벡터 인덱스 지정
    index=vector_index,
    # 유사한 항목 수 지정
    similarity_top_k=3,
)


In [ ]:
nodes = retriever.retrieve("Explain what OpenAI Sora model is.")

for node in nodes:
    print(node.text)
    print("===\n\n")

### Doc summary index

In [ ]:
# 문서 요약해주는 클래스
from llama_index.core import DocumentSummaryIndex
from llama_index.embeddings.openai import OpenAIEmbedding

# 모델 지정
chatgpt = OpenAIEmbedding(model = "text-embedding-3-small", metadata = None)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    llm=chatgpt,
    show_progress=True,
)

In [ ]:
doc_summary_index.get_document_summary(documents[0].id_)

In [ ]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)
retriever = DocumentSummaryIndexLLMRetriever(
    index=doc_summary_index,
    choice_batch_size=3,
    similarity_top_k=3,

)

retriever.retrieve("Mistarl AI는 어느 국가에 속해 있어?")

In [ ]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
)
retriever.retrieve("Mistarl AI는 어느 국가에 속해 있어?")

### Generator

In [ ]:
from llama_index.core import get_response_synthesizer

# 간결한 응답 요청
response_synthesizer = get_response_synthesizer(response_mode="compact")

response = response_synthesizer.synthesize(
    "OpenAI의 Sora 모델에 대해서 설명해줘.", nodes=retriever.retrieve("OpenAI의 Sora 모델에 대해서 설명해줘.")
)
response.response

In [ ]:
# 자세한 응답 요청
response_synthesizer = get_response_synthesizer(response_mode="accumulate")

response = response_synthesizer.synthesize(
    "OpenAI의 Sora 모델에 대해서 설명해줘.", nodes=retriever.retrieve("OpenAI의 Sora 모델에 대해서 설명해줘.")
)
response.response

In [ ]:
# 간단한 요약 응답 요청
response_synthesizer = get_response_synthesizer(response_mode="simple_summarize")

response = response_synthesizer.synthesize(
    "OpenAI의 Sora 모델에 대해서 설명해줘.", nodes=retriever.retrieve("OpenAI의 Sora 모델에 대해서 설명해줘.")
)
response.response

In [ ]:
# 계층적 요약 응답 요청
# 긴 텍스트를 처리하기 위해서
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")

response = response_synthesizer.synthesize(
    "OpenAI의 Sora 모델에 대해서 설명해줘.", nodes=retriever.retrieve("OpenAI의 Sora 모델에 대해서 설명해줘.")
)
response.response

In [ ]:
response_synthesizer.get_prompts()

### Manage prompt

In [ ]:
prompts_dict = query_engine.get_prompts()

prompts_dict

In [ ]:
for prompt in prompts_dict.values():
    print(prompt.get_template())
    print("***********\n\n")

In [ ]:
from llama_index.core import PromptTemplate


new_prompt_qa = """
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Use proper Korean, and answer in the style of kindergarten teacher, gentle and enthusiastic.
Query: {query_str}
Answer:
"""

new_prompt_refine = """
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Use proper Korean, and answer in the style of kindergarten teacher, gentle and enthusiastic.
Refined Answer:
"""

new_prompt_qa = PromptTemplate(new_prompt_qa)
new_prompt_refine = PromptTemplate(new_prompt_refine)


query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_prompt_qa, "response_synthesizer:refine_template": new_prompt_refine}
)

In [ ]:
query_engine.get_prompts()

In [ ]:
response = query_engine.query("MistralAI에 대해서 설명해줘")
response.response